In [1]:
my_list = [1, 3, 5, 2, 4, 9, 5, 8, 0, 7, 6]

In [2]:
def oracle(my_input):
    winner = 7
    return winner == my_input

In [3]:
for index, number in enumerate(my_list):
    if oracle(number):
        print('Winner found at index %i' %index)
        print('%i calls to the Oracle used' %(index + 1))
        break

Winner found at index 9
10 calls to the Oracle used


In [4]:
from qiskit import *
import matplotlib.pyplot as plt
import numpy as np

In [5]:
oracle = QuantumCircuit(2, name = 'oracle')
oracle.cz(0, 1)
oracle.to_gate()
oracle.draw()

q_0: ─■─
      │ 
q_1: ─■─

In [6]:
backend = Aer.get_backend('statevector_simulator')
grover_circ = QuantumCircuit(2, 2)
grover_circ.h([0, 1])
grover_circ.append(oracle, [0, 1])
grover_circ.draw()

┌───┐┌─────────┐
q_0: ┤ H ├┤0        ├
     ├───┤│  oracle │
q_1: ┤ H ├┤1        ├
     └───┘└─────────┘
c: 2/════════════════

In [7]:
job = execute(grover_circ, backend)
result = job.result()

In [8]:
sv = result.get_statevector()
np.around(sv, 2)

array([ 0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j])

In [11]:
reflection = QuantumCircuit(2, name = 'reflection')
reflection.h([0, 1])
reflection.z([0, 1])
reflection.cz(0, 1)
reflection.h([0, 1])
reflection.to_gate()

Instruction(name='reflection', num_qubits=2, num_clbits=0, params=[])

In [12]:
reflection.draw()

┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ Z ├─■─┤ H ├
     ├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤ Z ├─■─┤ H ├
     └───┘└───┘   └───┘

In [13]:
backend = Aer.get_backend('qasm_simulator')
grover_circ = QuantumCircuit(2, 2)
grover_circ.h([0, 1])
grover_circ.append(oracle, [0, 1])
grover_circ.append(reflection, [0, 1])
grover_circ.measure([0, 1], [0, 1])

In [14]:
grover_circ.draw()

┌───┐┌─────────┐┌─────────────┐┌─┐   
q_0: ┤ H ├┤0        ├┤0            ├┤M├───
     ├───┤│  oracle ││  reflection │└╥┘┌─┐
q_1: ┤ H ├┤1        ├┤1            ├─╫─┤M├
     └───┘└─────────┘└─────────────┘ ║ └╥┘
c: 2/════════════════════════════════╩══╩═
                                     0  1

In [15]:
job = execute(grover_circ, backend, shots = 1)
result = job.result()
result.get_counts()

{'11': 1}